In [1]:
from selenium import webdriver
import sys
import time
#sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException


## MAIN

antes de correr debes descargar el chrome driver acorde a tu maquina
https://sites.google.com/chromium.org/driver/downloads


In [2]:
path = "/opt/homebrew/bin/chromedriver"
keyword = "data science"
num_jobs = 15
slp_time = 5



'''Gathers jobs as a dataframe, scraped from Glassdoor'''
#Initializing the webdriver
options = webdriver.ChromeOptions()

#Uncomment the line below if you'd like to scrape without a new Chrome window every time.
#options.add_argument('headless')

#Change the path to where chromedriver is in your home folder.
driver = webdriver.Chrome(executable_path=path, options=options)
#driver.set_window_size(1120, 1000)

url = "https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType="
#url = 'https://www.glassdoor.com/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
driver.get(url)
time.sleep(5)
jobs = []


/var/folders/n2/np9y87ks47j0gqm4v82gvddh0000gn/T/ipykernel_57094/1339752204.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=path, options=options)


In [3]:
job_buttons = driver.find_elements(By.CLASS_NAME, "react-job-listing")  #jl for Job Listing. These are the buttons we're going to click.


In [4]:
def get_companyname_rating(company_name):
    """gets the company name and (if exist) gets the company rating too"""
    splited = company_name.split("\n")
    #company name comes with rating 
    if len(splited) > 1:
        return splited[0],  splited[1]
    #just company name
    else:
        return company_name, -1


In [5]:
job_buttons[1].click()
time.sleep(4)
#close modal
try:
    driver.find_element(By.XPATH, '//*[@id="JAModal"]/div/div[2]/span').click()
except NoSuchElementException:
    print("no hay modal xd")


collected_successfully = False

company_name_ = driver.find_element(By.CLASS_NAME, 'e1tk4kwz5').text
company_name, rating = get_companyname_rating(company_name_) 

location = driver.find_element(By.CLASS_NAME,'e1tk4kwz1').text
job_title = driver.find_element(By.CLASS_NAME,'e1tk4kwz2').text
job_description = driver.find_element(By.XPATH,'.//div[@class="jobDescriptionContent desc"]').text

salary_estimate = driver.find_element(By.CLASS_NAME,'e2u4hf18').text

collected_successfully = True



In [15]:

job_data_dict = {"Job Title" : job_title,
    "Company Name" : company_name,
     "Location" : location,
    "Salary Estimate" : salary_estimate,
    "Rating" : rating,
    "Job Description" : job_description,
   }

job_data_dict

{'Job Title': 'Jr Data Scientist (New York)',
 'Company Name': 'Paramount',
 'Location': 'New York, NY',
 'Salary Estimate': '$115,886 /yr (est.)',
 'Rating': '3.8',
 'Job Description': "Paramount Global (NASDAQ: PARA, PARAA) is a leading global media and entertainment company that creates premium content and experiences for audiences worldwide. Driven by iconic studios, networks and streaming services, Paramount's portfolio of consumer brands includes CBS, Showtime Networks, Paramount Pictures, Nickelodeon, MTV, Comedy Central, BET, Paramount+, Pluto TV and Simon & Schuster, among others. Paramount delivers the largest share of the U.S. television audience and boasts one of the industry's most important and extensive libraries of TV and film titles. In addition to offering innovative streaming services and digital video products, the company provides powerful capabilities in production, distribution and advertising solutions.\nAre you passionate about using data to help drive insights

In [17]:
Company_Overview = driver.find_elements(By.CLASS_NAME,'e1pvx6aw0')
def get_company_overview_data(Company_Overview):
    Company_Overview_dict = {"size":"",
                            "founded":"",
                            "type":"",
                            "industry":"",
                            "sector":"",
                            "revenue":"",}

    for i, key in enumerate(Company_Overview_dict):
        try: 
            Company_Overview_dict[key] = Company_Overview[i].text
        except:
            print(f"{key} not found")
            Company_Overview_dict[key] = -1
    return Company_Overview_dict
company_overview_dict = get_company_overview_data(Company_Overview)
company_overview_dict

In [18]:

job_data_dict.update(company_overview_dict)
    


In [ ]:
jobs.append(job_data_dict)

In [10]:
driver.close()